## Named Tuple - modifikace a rozšíření

In [134]:
from collections import namedtuple

In [135]:
Point2D = namedtuple("Poin2D", "x y")

In [136]:
pt = Point2D(10,20)

In [137]:
pt

Poin2D(x=10, y=20)

In [138]:
pt[0], pt[1] #pouze opáčko

(10, 20)

In [140]:
pt.x = 100

AttributeError: can't set attribute

Ale přiřazovat hodnoty nemůžeme takže jak to obejdeme?

In [141]:
id(pt)

2613156045952

In [143]:
pt = Point2D(100, pt.y)

In [144]:
pt

Poin2D(x=100, y=20)

In [145]:
id(pt)

2613156118144

Tzn. v podstatě abych něco modifikoval tak jedině tak, že přepíšu jeho hodnoty, ztratím ale původní referenci a získám novou, otázka jestli to není problém?

In [146]:
Stock = namedtuple("Stock", "symbol year month day open high low close")

In [147]:
djia = Stock("DJIA", 2018, 1, 25, 26_313, 26_458, 26_260, 26_393)

In [148]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

In [150]:
djia = Stock(djia.symbol, djia.year, djia.month, djia.day, djia.open, 
             djia.high, djia.low, 1000)

In [151]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=1000)

No to je prostě strašná blbost, abych změnil jednu hodnotu - tím že tvořím nový namedtuple jen s mírnou změnou, je ke zbláznění :D.

In [152]:
*values, close = djia

In [153]:
values, close

(['DJIA', 2018, 1, 25, 26313, 26458, 26260], 1000)

Takže pomocí unpackingu je to o něco rychlejší :).

In [155]:
values.append(26_393) #takže si poté přidám co chci - a dotvořím si namedtuple takto

In [156]:
djia = Stock(*values) #můj stock očekává 7 údajů proto je musím unpacknout

In [157]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=26393)

Takže jeden způsob jak urychlit přehnaný typing je pomocí packing a unpacking - pozor, že dostávám list při unpacakingu. Reference tam lítají pěkně divoce. Pokud chci přidávat jednu hodnotu je lepší .extend a ne append - zůstane mi stejná reference.

Též vše můžeme řešit pomocí slicingu.

In [158]:
djia = Stock(djia.symbol, djia.year, djia.month, djia.day, djia.open, 
             djia.high, djia.low, 1000)

In [159]:
values = djia[:7]

In [161]:
values #důležité je že pokud provedeme slicing zůstane nám to co máme, takže tuple

('DJIA', 2018, 1, 25, 26313, 26458, 26260)

In [162]:
#teď musíme samozřejmě příčíst další hodnotu a tedy vytvořit nový tuple
values + (100,) #čárku protože spojuji dva tuply?

('DJIA', 2018, 1, 25, 26313, 26458, 26260, 100)

In [163]:
djia =  Stock(*values, 1000) #nebo též takto - shodné jako u listů

In [164]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=1000)

Stále to jsou mizerné metody, pokud bychom chtěli něco měnit uprostřed. 

In [166]:
id(djia)

2613156069040

In [168]:
djia = djia._replace(year=2020, open=10000)

In [169]:
djia

Stock(symbol='DJIA', year=2020, month=1, day=25, open=10000, high=26458, low=26260, close=1000)

In [170]:
id(djia)

2613156065904

Takže o co zde jde? Máme skvělou metodu pro změnu údajů v tuplu, pomocí jeho metody replace. Pouze zadáme Co za Co --> ale stále tvoříme nový tuple, jiné id.

In [171]:
djia = Stock._make(values + (100,))
#_make metoda v pdstatě použije unpacking na values nic víc 

In [172]:
djia

Stock(symbol='DJIA', year=2018, month=1, day=25, open=26313, high=26458, low=26260, close=100)

Unpacking je podle mě hezčí, navíc se jedná o obecnou logiku, která se používá všude.

In [173]:
Point2D._fields

('x', 'y')

Budeme řešit v pdostatě to, že chceme z 2D udělat 3D tzn. přidat mu pole. Bez toho abych vše psal znovu. Jen doappendíme..

In [176]:
Point2D._fields + ("z",)

('x', 'y', 'z')

In [177]:
Poiunt3D = namedtuple("Point3D", Point2D._fields + ("z",))

In [180]:
Poiunt3D._fields

('x', 'y', 'z')

V podstatě nic extra, jde jen o to, že pokud budeme mít opravdu rozsáhlý namedtuple s hodně poli tak bude nejvhodnější použít jeho pole a přičíst nová pole.